In [2]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\Kevin\Downloads\openslide-win64-20221217\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm

In [3]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

In [ ]:
for idx, src in tqdm(enumerate(US_desired_src),total=len(US_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    print(num_rows)
    print(num_cols)
        # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src,imgname)
            Image.fromarray(nsimage).save(tile_save_src)

### Parallel processing:

In [1]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import pathos
from pathos.multiprocessing import ProcessPool
num_nodes = pathos.helpers.cpu_count()

In [3]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US2'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_3.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_4.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_5.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_6.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_7.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_8.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_9.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_3

In [4]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [4]:
def tile_wsi(src, wsi_name, save_src):
    import os
    import numpy as np
    from PIL import Image
    import math
    import pandas as pd
    from tqdm import tqdm
    from pathos.multiprocessing import ProcessPool
    OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src, imgname)
            Image.fromarray(nsimage).save(tile_save_src)

In [ ]:
# started running this on 2:50 PM 4/30/23
# for US:
pool = ProcessPool(nodes=num_nodes)
results = []

for idx, src in tqdm(enumerate(US_desired_src), total=len(US_desired_src), colour='red', desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src, wsi_name)
    results.append(pool.apipe(tile_wsi, src, wsi_name, save_src))

# Get the results
for result in results:
    import os
    result.get()

# Close the process pool
pool.close()

WSI Processed: 100%|██████████| 25/25 [00:00<?, ?it/s]


### Same code for HE:

In [1]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import pathos
from pathos.multiprocessing import ProcessPool
num_nodes = pathos.helpers.cpu_count()

In [2]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]

HE_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\HE'
HE_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
HE_desired_src = [os.path.join(HE_ndpi_src,x) for x in he_desired_names_src]
HE_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_1_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_2_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_4_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_5_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_6_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_7_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_8_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_9_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_1_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_2_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unsta

In [3]:
def tile_wsi(src, wsi_name, save_src):
    import os
    import numpy as np
    from PIL import Image
    import math
    import pandas as pd
    from tqdm import tqdm
    from pathos.multiprocessing import ProcessPool
    OPENSLIDE_PATH = r"C:\Users\labadmin\Downloads\openslide-win64-20230414\bin" # this will change
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            heimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            heimage = np.array(heimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src, imgname)
            Image.fromarray(heimage).save(tile_save_src)

In [ ]:
# for HE:
pool = ProcessPool(nodes=num_nodes) #pathos.helpers.cpu_count()
results = []

for idx, src in tqdm(enumerate(HE_desired_src), total=len(HE_desired_src), colour='red', desc='WSI Processed'):
    wsi_name = he_desired_names[idx]
    save_src = os.path.join(HE_dst_src, wsi_name)
    results.append(pool.apipe(tile_wsi, src, wsi_name, save_src))

# Get the results
for result in results:
    import os
    result.get()

# Close the process pool
pool.close()

WSI Processed: 100%|██████████| 25/25 [00:00<00:00, 1587.26it/s]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\labadmin\PycharmProjects\wsi_analysis\venv\lib\site-packages\openslide\lowlevel.py", line 54, in <module>
    _lib = cdll.LoadLibrary('libopenslide-0.dll')
  File "C:\Users\labadmin\AppData\Local\Programs\Python\Python39\lib\ctypes\__init__.py", line 452, in LoadLibrary
    return self._dlltype(name)
  File "C:\Users\labadmin\AppData\Local\Programs\Python\Python39\lib\ctypes\__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
FileNotFoundError: Could not find module 'libopenslide-0.dll' (or one of its dependencies). Try using the full path with constructor syntax.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\labadmin\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\la